# 데이터프레임 합성
판다스는 두 개 이상의 데이터프레임을 하나로 합치는 데이터 병합(merge)이나 연결(concatenate)을 지원한다.

## merge

- merge 함수는 두 데이터 프레임의 공통 열 혹은 인덱스를 기준으로 두 개의 테이블을 합친다. 이 때 기준이 되는 열, 행의 데이터를 키(key)라고 한다.

In [1]:
import pandas as pd

In [2]:
df1 = pd.DataFrame({
    '계좌번호': [1001, 1002, 1003, 1004, 1005, 1006, 1007],
    '이름': ['둘리', '도우너', '또치', '길동', '희동', '마이콜', '영희']
}, columns=['계좌번호', '이름'])
df1

,계좌번호,이름
0,1001,둘리
1,1002,도우너
2,1003,또치
3,1004,길동
4,1005,희동
5,1006,마이콜
6,1007,영희


In [3]:
df2 = pd.DataFrame({
    '계좌번호': [1001, 1001, 1005, 1006, 1008, 1001],
    '금액': [10000, 20000, 15000, 5000, 100000, 30000]
}, columns=['계좌번호', '금액'])
df2

,계좌번호,금액
0,1001,10000
1,1001,20000
2,1005,15000
3,1006,5000
4,1008,100000
5,1001,30000


- merge 함수로 위의 두 데이터프레임 df1, df2 를 합치면 공통 열인 계좌번호 열을 기준으로 데이터를 찾아서 합친다.
- 이 때 기본적으로는 양쪽 데이터프레임에 모두 키가 '존재하는 데이터'만 보여주는 inner join 방식을 사용한다.

In [4]:
pd.merge(df1, df2) # default = inner join 교집합

,계좌번호,이름,금액
0,1001,둘리,10000
1,1001,둘리,20000
2,1001,둘리,30000
3,1005,희동,15000
4,1006,마이콜,5000


- outer join 방식은 키 값이 한쪽에만 있어도 데이터를 보여준다.

In [5]:
pd.merge(df1, df2, how='outer') # 합집합

,계좌번호,이름,금액
0,1001,둘리,10000.0
1,1001,둘리,20000.0
2,1001,둘리,30000.0
3,1002,도우너,NaN
4,1003,또치,NaN
5,1004,길동,NaN
6,1005,희동,15000.0
7,1006,마이콜,5000.0
8,1007,영희,NaN
9,1008,NaN,100000.0


left, right 방식은 각각 첫번째, 혹은 두번째 데이터프레임의 키 값을 모두 보여준다.

In [6]:
pd.merge(df1, df2, how='left') # left를 기준으로

,계좌번호,이름,금액
0,1001,둘리,10000.0
1,1001,둘리,20000.0
2,1001,둘리,30000.0
3,1002,도우너,NaN
4,1003,또치,NaN
5,1004,길동,NaN
6,1005,희동,15000.0
7,1006,마이콜,5000.0
8,1007,영희,NaN


In [7]:
pd.merge(df1, df2, how='right')

,계좌번호,이름,금액
0,1001,둘리,10000
1,1001,둘리,20000
2,1005,희동,15000
3,1006,마이콜,5000
4,1008,NaN,100000
5,1001,둘리,30000


만약 테이블에 키 값이 같은 데이터가 여러개 있는 경우에는 있을 수 있는 모든 경우의 수를 따져서 조합을 만들어 낸다.

In [8]:
df1 = pd.DataFrame({
    '품종': ['setosa', 'setosa', 'virginica', 'virginica'],
    '꽃잎길이': [1.4, 1.3, 1.5, 1.3]},
    columns=['품종', '꽃잎길이'])
df1

,품종,꽃잎길이
0,setosa,1.4
1,setosa,1.3
2,virginica,1.5
3,virginica,1.3


In [9]:
df2 = pd.DataFrame({
    '품종': ['setosa', 'virginica', 'virginica', 'versicolor'],
    '꽃잎너비': [0.4, 0.3, 0.5, 0.3]},
    columns=['품종', '꽃잎너비'])
df2

,품종,꽃잎너비
0,setosa,0.4
1,virginica,0.3
2,virginica,0.5
3,versicolor,0.3


이 데이터에서 키 값 setosa에 대해 왼쪽 데이터프레임(df1)는 1.4와 1.3라는 2개의 데이터,     
오른쪽 데이터프레임(df2)에 0.4라는 1개의 데이터가 있으므로 병합된 데이터에는 setosa가 (1.4, 0.4), (1.3, 0.4) 두 개의 데이터가 생긴다.     
키 값 virginica의 경우에는 왼쪽 데이터프레임에 1.5와 1.3라는 2개의 데이터,    
오른쪽 데이터프레임에 0.3와 0.5라는 2개의 데이터가 있으므로 2개와 2개의 조합에 의해 4가지 값이 생긴다.

In [10]:
pd.merge(df1, df2)

,품종,꽃잎길이,꽃잎너비
0,setosa,1.4,0.4
1,setosa,1.3,0.4
2,virginica,1.5,0.3
3,virginica,1.5,0.5
4,virginica,1.3,0.3
5,virginica,1.3,0.5


두 데이터프레임에서 이름이 같은 열은 모두 키가 된다.   
만약 이름이 같아도 키가 되면 안되는 열이 있다면 on 인수로 기준열을 명시해야 한다.   
다음 예에서 첫번째 데이터프레임의 "데이터"는 실제로는 금액을 나타내는 데이터이고   
두번째 데이터프레임의 "데이터"는 실제로는 성별을 나타내는 데이터이므로 이름이 같아도 다른 데이터이다.   
따라서 이 열은 기준열이 되면 안된다.

In [11]:
df1 = pd.DataFrame({
    '고객명': ['춘향', '춘향', '몽룡'],
    '날짜': ['2018-01-01', '2018-01-02', '2018-01-01'],
    '데이터': ['20000', '30000', '100000']})
df1

,고객명,날짜,데이터
0,춘향,2018-01-01,20000
1,춘향,2018-01-02,30000
2,몽룡,2018-01-01,100000


In [12]:
df2 = pd.DataFrame({
    '고객명': ['춘향', '몽룡'],
    '데이터': ['여자', '남자']})
df2

,고객명,데이터
0,춘향,여자
1,몽룡,남자


In [13]:
pd.merge(df1, df2) # 고객명과 데이터칼럼의 &조건으로 교집합 (데이터라는 열이 가격과 성별 모두 포함하고 있기때문에)

,고객명,날짜,데이터


In [14]:
pd.merge(df1, df2, on='고객명') # 이 때 기준 열이 아니면서 이름이 같은 열에는 _x 또는 _y 와 같은 접미사가 붙는다.

,고객명,날짜,데이터_x,데이터_y
0,춘향,2018-01-01,20000,여자
1,춘향,2018-01-02,30000,여자
2,몽룡,2018-01-01,100000,남자


반대로 키가 되는 기준열의 이름이 두 데이터프레임에서 다르다면 left_on, right_on 인수를 사용하여 기준열을 명시해야 한다.

In [15]:
df1 = pd.DataFrame({
    '이름': ['영희', '철수', '철수'],
    '성적': [1, 2, 3]})
df1

,이름,성적
0,영희,1
1,철수,2
2,철수,3


In [16]:
df2 = pd.DataFrame({
    '성명': ['영희', '영희', '철수'],
    '성적2': [4, 5, 6]})
df2

,성명,성적2
0,영희,4
1,영희,5
2,철수,6


In [17]:
pd.merge(df1, df2, left_on='이름', right_on="성명")

,이름,성적,성명,성적2
0,영희,1,영희,4
1,영희,1,영희,5
2,철수,2,철수,6
3,철수,3,철수,6


## join 메서드
merge는 결합 기준이 열, join은 결합 기준이 인덱스임

In [18]:
df1.join(df2, how='outer')

,이름,성적,성명,성적2
0,영희,1,영희,4
1,철수,2,영희,5
2,철수,3,철수,6


### Quiz. 두 개의 데이터프레임을 만들고 merge 명령으로 합친다. 단 데이터프레임은 다음 조건을 만족해야 한다.
```
1. 각각 5 x 5 이상의 크기를 가진다.
2. 공통 열을 하나 이상 가진다. 다만 공통 열의 이름은 서로 다르다.
```

In [19]:
import numpy as np
# 공통열 만들기
np.array(list(range(1, 6))).reshape(5,1)

array([[1],
       [2],
       [3],
       [4],
       [5]])

In [20]:
# 벨류 만들기
np.random.randint(0, 101, size=(5, 4))

array([[99, 34, 23, 59],
       [17, 62, 51, 62],
       [50, 52, 75, 91],
       [ 6, 16, 86, 18],
       [37, 54, 55, 28]])

In [21]:
# 첫 번째 데이터프레임 생성
df1 = pd.DataFrame( np.hstack([np.array(list(range(1, 6))).reshape(5,1), 
                              np.random.randint(0, 101, size=(5, 4))]),
                   columns=['A', 'B', 'C', 'D', 'E'])
df1

,A,B,C,D,E
0,1,26,72,87,44
1,2,54,93,89,51
2,3,86,7,32,42
3,4,94,75,29,88
4,5,39,12,22,90


In [22]:
# 두 번째 데이터프레임 생성
df2 = pd.DataFrame(np.hstack([np.array(list(range(1, 6))).reshape(5,1), 
                              np.random.randint(0, 101, size=(5, 4))]),
                              columns=['A1', 'B1', 'C1', 'D1', 'E1'])
df2

,A1,B1,C1,D1,E1
0,1,72,50,84,81
1,2,90,52,62,76
2,3,14,12,38,22
3,4,80,7,0,52
4,5,9,35,44,74


In [23]:
# 두 데이터프레임을 merge 명령으로 합침
df_merged = pd.merge(df1, df2, left_on='A', right_on='A1')
df_merged

,A,B,C,D,E,A1,B1,C1,D1,E1
0,1,26,72,87,44,1,72,50,84,81
1,2,54,93,89,51,2,90,52,62,76
2,3,86,7,32,42,3,14,12,38,22
3,4,94,75,29,88,4,80,7,0,52
4,5,39,12,22,90,5,9,35,44,74


## concat 함수를 사용한 데이터 연결
`concat` 함수를 사용하면 기준 열(key column)을 사용하지 않고 단순히 데이터를 연결(concatenate)한다. 기본적으로는 위/아래로 데이터 행을 연결한다. 단순히 두 시리즈나 데이터프레임을 연결하기 때문에 인덱스 값이 중복될 수 있다.



In [24]:
s1 = pd.Series([0, 1], index=['A', 'B'])
s1

A    0
B    1
dtype: int64

In [25]:
s2 = pd.Series([2, 3, 4], index=['A', 'B', 'C'])
s2

A    2
B    3
C    4
dtype: int64

In [26]:
pd.concat([s1, s2])

A    0
B    1
A    2
B    3
C    4
dtype: int64

In [27]:
df1 = pd.DataFrame(
    np.arange(6).reshape(3, 2),
    index=['a', 'b', 'c'],
    columns=['데이터1', '데이터2'])
df1

,데이터1,데이터2
a,0,1
b,2,3
c,4,5


In [28]:
df2 = pd.DataFrame(
    5 + np.arange(4).reshape(2, 2),
    index=['a', 'c'],
    columns=['데이터3', '데이터4'])
df2

,데이터3,데이터4
a,5,6
c,7,8


In [29]:
pd.concat([df1, df2], axis=1)

,데이터1,데이터2,데이터3,데이터4
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


### Quiz. 실적데이터
어느 회사의 전반기(1월 ~ 6월) 실적을 나타내는 데이터프레임과 후반기(7월 ~ 12월) 실적을 나타내는 데이터프레임을 만든 뒤 합친다.   
실적 정보는 "매출", "비용", "이익" 으로 이루어진다. (이익 = 매출 - 비용).   
또한 1년간의 총 실적을 마지막 행으로 덧붙인다.

In [33]:
data1 = np.random.randint(0, 101, (6, 2))
df1 = pd.DataFrame(data1, columns=['매출', '비용'])
df1.index = ['1월', '2월', '3월', '4월', '5월', '6월']
df1["이익"] = df1.매출 - df1.비용
df1

,매출,비용,이익
1월,44,43,1
2월,57,32,25
3월,9,84,-75
4월,1,36,-35
5월,6,0,6
6월,49,54,-5


In [34]:
data2 = np.random.randint(0, 101, (6, 2))
df2 = pd.DataFrame(data1, columns=['매출', '비용'])
df2.index = ['7월', '8월', '9월', '10월', '11월', '12월']
df2["이익"] = df2.매출 - df2.비용
df2

,매출,비용,이익
7월,44,43,1
8월,57,32,25
9월,9,84,-75
10월,1,36,-35
11월,6,0,6
12월,49,54,-5


In [35]:
df = pd.concat([df1, df2], axis=0)
df

,매출,비용,이익
1월,44,43,1
2월,57,32,25
3월,9,84,-75
4월,1,36,-35
5월,6,0,6
6월,49,54,-5
7월,44,43,1
8월,57,32,25
9월,9,84,-75
10월,1,36,-35


In [36]:
df.loc['총실적'] = df.sum(axis=0)
df

,매출,비용,이익
1월,44,43,1
2월,57,32,25
3월,9,84,-75
4월,1,36,-35
5월,6,0,6
6월,49,54,-5
7월,44,43,1
8월,57,32,25
9월,9,84,-75
10월,1,36,-35
